# Kinetics Estimator

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

import rmgpy
from rmgpy.data.kinetics.family import TemplateReaction
from rmgpy.data.kinetics.depository import DepositoryReaction
from rmgpy.data.rmg import RMGDatabase
from rmgpy.kinetics.kineticsdata import KineticsData
from rmgpy.molecule.molecule import Molecule
from rmgpy.reaction import Reaction
from rmgpy.rmg.model import CoreEdgeReactionModel
from rmgpy.species import Species
from rmgpy.thermo.thermoengine import submit

## Set reaction families of interest

In [ ]:
families = ['R_Addition_MultipleBond']

## Load database

In [ ]:
database_path = rmgpy.settings['database.directory']

database = RMGDatabase()
database.load(
    path = database_path,
    thermo_libraries = ['primaryThermoLibrary'],
    reaction_libraries = [],
    seed_mechanisms = [],
    kinetics_families = families,
)

# Load training data
for family in database.kinetics.families.values():
    family.add_rules_from_training(thermo_database=database.thermo)
    family.fill_rules_by_averaging_up(verbose=True)

## Set reactants and products

In [ ]:
# If you only want to specify reactants, just run this block
reactants = [
    Species(smiles='c1ccccc1C=C'),
    Species(smiles='[CH3]')
]
products = None
for r in reactants:
    submit(r)
    display(r)

In [ ]:
# If you also want to specify products, run this block as well
products = [
    Species(smiles="c1ccccc1[CH]CC"),
]
for p in products:
    submit(p)
    display(p)

## Generate reactions

In [ ]:
reaction_list = database.kinetics.generate_reactions_from_families(
    reactants,
    products=products,
    only_families=None,
    resonance=True,
)

## Process reactions and apply kinetics

In [ ]:
cerm = CoreEdgeReactionModel()
for rxn0 in reaction_list:
    rxn1 = cerm.make_new_reaction(rxn0)
for rxn0 in cerm.new_reaction_list:
    cerm.apply_kinetics_to_reaction(rxn0)
    if isinstance(rxn0.kinetics, KineticsData):
        rxn0.kinetics = reaction.kinetics.to_arrhenius()
    if isinstance(rxn0, (TemplateReaction, DepositoryReaction)):
        rxn0.fix_barrier_height()

## Display results

In [ ]:
pressure = 1e5 # Pa
temperature = np.linspace(298, 2000, 50)

In [ ]:
def plot_kinetics(reaction):

    fig = plt.figure()

    kinetics = reaction.kinetics
    conversion_factor = kinetics.A.get_conversion_factor_from_si_to_cm_mol_s()
    if len(reaction.reactants) == 1:
        kunits = 's^-1'
    elif len(reaction.reactants) == 2:
        kunits = 'cm^3/(mol*s)'
    else:
        kunits = '???'

    k = []
    for t in temperature:
        # Rates are returned in SI units by default
        k.append(conversion_factor * kinetics.get_rate_coefficient(t, pressure))

    x = 1000 / temperature

    plt.semilogy(x, k)

    plt.xlabel('1000/T (K)')
    plt.ylabel('k [{0}]'.format(kunits))
    plt.legend([str(reaction)], loc=8, bbox_to_anchor=(0.5, 1.02))
    
    plt.show()

In [ ]:
for rxn0 in cerm.new_reaction_list:
    display(rxn0)
    print(rxn0.kinetics)
    plot_kinetics(rxn0)

## Calculate equilibrium constants and reverse rate coefficients

In [ ]:
# Pick a single reaction
selected_rxn = cerm.new_reaction_list[0]
display(selected_rxn)

In [ ]:
keq = selected_rxn.get_equilibrium_constants(temperature)

fig = plt.figure()
plt.semilogy(temperature, keq)
plt.semilogy([298, 2000], [1, 1], 'r--')
plt.xlabel('Temperature (K)')
plt.ylabel('Keq')
plt.legend([str(selected_rxn)], loc=8, bbox_to_anchor=(0.5, 1.02))
plt.show()

In [ ]:
reverse_rxn = Reaction(
    reactants=selected_rxn.products,
    products=selected_rxn.reactants,
    kinetics=selected_rxn.generate_reverse_rate_coefficient()
)
display(reverse_rxn)
print(reverse_rxn.kinetics)
plot_kinetics(reverse_rxn)